In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# plotting 
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)
#plt.rcParams.update({'font.size': 12})
import os
#!pip install dabl > /dev/null
# !pip install datatable > /dev/null
#import dabl
import datatable as dt
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
# for the image import
import os
from IPython.display import Image
# garbage collector to keep RAM in check
import gc
from tqdm import tqdm
from scipy import stats

from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
import optuna

In [ ]:
# List Of Files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%%time

s = !wc -l {'/kaggle/input/jane-street-market-prediction/train.csv'}
n_train_rows = int(s[0].split(' ')[0])+1
print("Total Number Of Rows: ", n_train_rows)
del s
del n_train_rows

### Data Load Into Dataframe
1. Training data is very large in size (almost 2GB+ )
2. DataTable is more faster than pandas dataframe

In [ ]:
# %%time
# Data Load For Analysis
# For Faster load, use datatable(R) which is faster than pandas
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()
print(train_data.info())
# Delete unnecessary variable in order to free more space from RAM
del train_data_datatable

### Basic Exploratory Data Analysis (EDA)

In [ ]:
train_data.head(2)

In [ ]:
print(train_data.columns.to_list())

In [ ]:
print("Total Unique Date : ",train_data['date'].nunique())
print("Row Count", train_data.shape)
print('-'*30)
print("Column Type : ")
print(f"{train_data.dtypes.value_counts()}")

* Training data has 130 attributes/feature.
* Total unique date 500.
* The date seem to contain 2 years of trading data since the trading days of the year are approximately 252 : 253 days

In [ ]:
# # Missing Data Percentage Count
# n_features = 50
# missing_values_count = train_data.isnull().sum()[train_data.isna().sum() > 0].sort_values(ascending=False)
# # print (missing_values_count)
# total_cells = np.product(train_data.shape)
# total_missing = missing_values_count.sum()
# print('NAN Valued Columns: %d' %train_data.isna().any().sum())
# print ("Missing data = ",str(round((total_missing/total_cells) * 100, 2))+'%')

# fig, axs = plt.subplots(figsize=(10, 10))

# sns.barplot(y = missing_values_count.index[0:n_features], 
#             x = missing_values_count.values[0:n_features], 
#             alpha = 1.0
#            )

# plt.title(f'NaN values of train dataset (Top {n_features})')
# plt.xlabel('NaN values')
# fig.savefig(f'nan_values_top_{n_features}_features.png')
# plt.show()

# # Delete unnecessary variable in order to free more space from RAM
# del missing_values_count
# del total_cells
# del total_missing

* Lot's of null value(almost 2%), may need to remove or replace with other value

In [ ]:
# Check if there are any unnecessary variable or not
#%whos

#### RESP & Weight EDA

In [ ]:
# plt.figure(1, figsize = (15 , 10))
# j=0
# resp_list = ['resp','resp_1','resp_2','resp_3','resp_4']
# for i in [[x,y] for x in [0,1,2] for y in [0,1]]:
    
#     if i == [0,0]:
#         train_data['trend'] = pd.Series(train_data[resp_list[j]]).cumsum()                                                              
#         train_data['weighted_trend'] = pd.Series(train_data['weight'] * train_data[resp_list[j]]).cumsum()
#         plt.subplot2grid((3 , 2) , (0, 0), colspan=2)
#         plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
#         plt.plot(train_data['ts_id'], train_data['trend'])
#         plt.plot(train_data['ts_id'], train_data['weighted_trend'])
#         gc.collect();
#         # Delete New Created Columns 
#         train_data = train_data.drop(columns=['trend', 'weighted_trend'], axis=1)
        
#         j=j+1
#     elif i == [0,1]: 
#         pass
#     else:
#         train_data['trend'] = pd.Series(train_data[resp_list[j]]).cumsum()                                                              
#         train_data['weighted_trend'] = pd.Series(train_data['weight'] * train_data[resp_list[j]]).cumsum()
#         plt.subplot2grid((3 , 2) , (i[0], i[1]), colspan=1)
#         plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
#         plt.plot(train_data['ts_id'], train_data['trend'])
#         plt.plot(train_data['ts_id'], train_data['weighted_trend'])
#         gc.collect();
#         # Delete New Created Columns 
#         train_data = train_data.drop(columns=['trend', 'weighted_trend'], axis=1)
#         j = j+1
# plt.tight_layout()
# plt.show()

* The overall market is going up
* The weighted returns are trending downwards so you would want to predict fewer ones for this time period.

In [ ]:
# fig = plt.figure(figsize=(16,6))
# ax = plt.subplot(1,1,1)
# train_data.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']].sum().cumsum().plot(ax=ax)
# plt.title('Cumulative Sum of Different RESP\'s',fontsize=18)
# plt.xlabel('Date',fontsize=14)
# plt.axvspan(0,85,linestyle=':',linewidth=2,label='first 92 days',color='darkorange',alpha=.2)
# plt.legend(fontsize=12,ncol=3,loc='lower right');

* more gains in the first 92 days :->  it may be a good idea to drop the observations before this point,
* resp_4 has the highest cumulative sum.
* resp_1 has the smallest cumulative sum.

In [ ]:
# train_data['weight_resp']   = train_data['weight']*train_data['resp']
# train_data['weight_resp_1'] = train_data['weight']*train_data['resp_1']
# train_data['weight_resp_2'] = train_data['weight']*train_data['resp_2']
# train_data['weight_resp_3'] = train_data['weight']*train_data['resp_3']
# train_data['weight_resp_4'] = train_data['weight']*train_data['resp_4']

# fig, ax = plt.subplots(figsize=(15, 5))
# resp    = pd.Series(1+(train_data.groupby('date')['weight_resp'].mean())).cumprod()
# resp_1  = pd.Series(1+(train_data.groupby('date')['weight_resp_1'].mean())).cumprod()
# resp_2  = pd.Series(1+(train_data.groupby('date')['weight_resp_2'].mean())).cumprod()
# resp_3  = pd.Series(1+(train_data.groupby('date')['weight_resp_3'].mean())).cumprod()
# resp_4  = pd.Series(1+(train_data.groupby('date')['weight_resp_4'].mean())).cumprod()
# ax.set_xlabel ("Day", fontsize=18)
# ax.set_title ("Cumulative daily return for resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
# resp.plot(lw=3, label='resp x weight')
# resp_1.plot(lw=3, label='resp_1 x weight')
# resp_2.plot(lw=3, label='resp_2 x weight')
# resp_3.plot(lw=3, label='resp_3 x weight')
# resp_4.plot(lw=3, label='resp_4 x weight')
# # day 85 marker
# ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
# ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
# plt.legend(loc="lower left");
# # Delete Unnecessary Variables
# train_data = train_data.drop(columns=['weight_resp','weight_resp_1','weight_resp_2', 'weight_resp_3','weight_resp_4'], axis=1)
# del resp, resp_1, resp_2, resp_3, resp_4

* We can see that the shortest time horizons, resp_1, resp_2 and resp_3, representing a more conservative strategy, result in the lowest return.

In [ ]:
# fig = px.line(train_data.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4','resp']].mean(),
#               x= train_data.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4','resp']].mean().index,
#               y= ['resp_1', 'resp_2', 'resp_3', 'resp_4','resp'],
#               title= 'Average Resp per day')
# fig.layout.xaxis.title = 'Day' 
# fig.layout.yaxis.title = 'Avg Resp'
# fig.show()

In [ ]:
def highlight(x):
    df = x.copy()
    df.loc[:,:] = '' 
    df.iloc[::,2] = 'background-color: yellow'
    return df

temp = train_data[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']].describe().T
temp.style.apply(highlight, axis = None)

* from standard deviation we can assume that **resp** is more related to longer time horizon invest as longer time horizon are associated with more return and higher risk

In [ ]:
# sns.pairplot(train_data[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']], corner=True )

* **resp** is highly related to **resp_4 **
* **resp_1** and **resp_2** are highly relted

### Investment Time Horizon
*An Investment Time Horizon is the period where one expects to hold an investment for a specific goal. Investments are generally broken down into two main categories: stocks (riskier) and bonds (less risky). The longer the Time Horizon, the more aggressive, or riskier portfolio, an investor can build. The shorter the Time Horizon, the more conservative, or less risky, the investor may want to adopt.*

In [ ]:
# fig = px.area(data_frame= train_data.groupby('date')[['resp']].count(),title='Number of operation per day')
# fig.update_traces(showlegend = False)
# fig.layout.xaxis.title = 'Day' 
# fig.layout.yaxis.title = 'Number of operations'
# fig.show()

In [ ]:
# fig = px.area(data_frame= train_data.groupby('date')[['resp']].sum(),title='Resp sum of operation per day')
# fig.update_traces( showlegend = False)
# fig.layout.xaxis.title = 'Day' 
# fig.layout.yaxis.title = 'Resp sum'
# fig.show()

In [ ]:
# plt.figure(figsize = (12,5))
# ax = sns.distplot(train_data['resp'], 
#              bins=3000, 
#              kde_kws={"clip":(-0.05,0.05)}, 
#              hist_kws={"range":(-0.05,0.05)},
#              color='darkcyan', 
#              kde=False);
# values = np.array([rec.get_height() for rec in ax.patches])
# norm = plt.Normalize(values.min(), values.max())
# colors = plt.cm.jet(norm(values))
# for rec, col in zip(ax.patches, colors):
#     rec.set_color(col)
# plt.xlabel("Histogram of the resp values", size=14)
# plt.show();
# gc.collect();

In [ ]:
print('The minimum value for resp is: %.5f' % train_data['resp'].min())
print('The maximum value for resp is:  %.5f' % train_data['resp'].max())
print("Skew of resp is: %.2f" %train_data['resp'].skew() )
print("Kurtosis of resp is: %.2f"  %train_data['resp'].kurtosis() )

Leptokartic Kurtosis (as kurtosis > 3)

In [ ]:
print('Percentage of zero weights is: %.2f' % (train_data[train_data['weight']==0]['weight'].count()/train_data.shape[0]*100) +"%")
print('The minimum weight is: %.2f' % train_data['weight'].min())
print('The maximum weight is: %.2f' % train_data['weight'].max())

* Each trade has an associated weight and resp, which together represents a return on the trade. 
* Trades with weight = 0 were intentionally included in the dataset for completeness, although such trades will not contribute towards the scoring evaluation.

In [ ]:
# date = 0
# n_features = 130

# cols = [f'feature_{i}' for i in range(1, n_features)]
# hist = px.histogram(
#     train_data[train_data["date"] == date], 
#     x=cols, 
#     animation_frame='variable', 
#     range_y=[0, 600], 
#     range_x=[-7, 7]
# )

# hist.show()

### Data Cleaning
##### Missing Value

In [ ]:
print(train_data.shape)

In [ ]:
# Replace NaN Value with Mean Value
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_train_df = pd.DataFrame(imputer.fit_transform(train_data))

imputed_train_df.columns = train_data.columns
imputed_train_df.index = train_data.index

print(f"Column With NaN Value : {imputed_train_df.isna().any().sum()}")
del train_data, imputer

In [ ]:
imputed_train_df.head()

In [ ]:
# # Remove Duplicate Value
# imputed_train_df.drop_duplicates(keep=False, inplace=True)
# print(imputed_train_df.shape)

In [ ]:
# Calculating the Z score : remove outliers

threshold = 4

z = np.abs(stats.zscore(imputed_train_df, nan_policy='omit'))
imputed_train_df = imputed_train_df[(z < threshold).all(axis=1)].reset_index(drop=True)

del z

In [ ]:
imputed_train_df['action'] = (imputed_train_df['resp'] > 0 ).astype('int')

In [ ]:
# Check target class is balanced or unbalanced in the training data

sns.countplot(x="action", data = imputed_train_df)

In [ ]:
X = imputed_train_df.loc[:, imputed_train_df.columns.str.contains('feature')]
y = imputed_train_df.loc[:, 'action']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
del imputed_train_df
# del X,y

In [ ]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest  = xgb.DMatrix(x_test, label=y_test)
del x_train
del x_test

In [ ]:
def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, .1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method' : 'gpu_hist',
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dtest)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=5)
del dtrain, dtest

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show();

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show();

In [ ]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
best_params = study.best_trial.params
# best_params['gpu_id'] = 0
best_params['tree_method'] = 'gpu_hist'
best_params['objective'] = 'binary:logistic'

del study

In [ ]:
# Fit the XGBoost classifier with optimal hyperparameters
clf = xgb.XGBClassifier(**best_params)

In [ ]:
%time clf.fit(X, y)  #Used the whole training data

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test()

In [ ]:
# th = 0.5
# for (test_df, pred_df) in tqdm(env.iter_test()):
#     if test_df['weight'].item() > 0:
#         x_tt = test_df.loc[:, test_df.columns.str.contains('feature')].values
#         if np.isnan(x_tt[:, 1:].sum()):
#             x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:])
#         pred = clf.predict(x_tt)
#         pred_df.action = np.where(pred >= th, 1, 0).astype(int)
#     else:
#         pred_df.action = 0
#     env.predict(pred_df)

In [ ]:
th = 0.5
for (test_df, pred_df) in tqdm(iter_test):
    if test_df['weight'].item() > 0:
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        y_preds = clf.predict(X_test)
        pred_df.action = np.where(y_preds >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

In [ ]:
pred_df.head()